In [ ]:
# Import all the relevant libraries and classes
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
!nvidia-smi

In [ ]:
# Import data from kaggle
train = pd.read_csv('../input/digit-recognizer/train.csv') 
test = pd.read_csv('../input/digit-recognizer/test.csv')

X_train = train.drop(['label'], axis = 1)
y_train = train['label']

X_test = test


# Divide the labels in the training, testing and validation set
y_train = tf.constant(y_train)


# Divide the input in the training, testing and validation set
X_train = tf.constant(np.array(X_train), dtype = tf.float64)
X_test = tf.constant(np.array(X_test), dtype = tf.float32)

In [ ]:
X_train.shape, X_test.shape

In [ ]:
type(X_test)

In [ ]:
plt.figure(figsize = (10, 10))
count = 1
idd = 0
for i in range(5):
    for j in range(5):
        plt.subplot(5, 5, count)
        plt.imshow(tf.constant(X_test[idd, :], shape = [28, 28]), cmap = 'gray')
        idd += 1
        count += 1

In [ ]:
# Scale the input
X_train_s = (X_train - 127.5) / 127.5
X_test_s = (X_test - 127.5) / 127.5

In [ ]:
X_train_s.shape, X_test_s.shape

In [ ]:
#Create the model

mod = keras.models.Sequential([
    
    keras.layers.InputLayer(input_shape = [784]), 
    
    keras.layers.Dense(576),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha = 0.4),
    
    keras.layers.Dense(320),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    
    keras.layers.Dense(256),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    
    keras.layers.Dense(160, kernel_regularizer = keras.regularizers.l2(0.0025)),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha = 0.4),
    
    keras.layers.Dense(84, kernel_regularizer = keras.regularizers.l2(0.0025)),
    keras.layers.BatchNormalization(),
    keras.layers.LeakyReLU(alpha = 0.25),
    
    keras.layers.Dense(10, activation = keras.activations.softmax),
])



# Compile the model
mod.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005), 
            loss = keras.losses.sparse_categorical_crossentropy, 
            metrics = ['accuracy'])



# summarize the model
mod.summary()


# Initialize callbacks
cb1 = EarlyStopping(patience = 3, restore_best_weights=True, monitor = 'val_acc')
cb2 = ReduceLROnPlateau(patience = 4, min_lr=0.00001, factor = 0.4, monitor = 'val_acc')

# Plot the model
# keras.utils.plot_model(mod)

In [ ]:
# Fit the data to the model and pass the validation data
history = mod.fit(X_train_s, y_train, validation_split=0.2, epochs = 475, batch_size = 140, callbacks = [cb1, cb2], verbose = 0)

In [ ]:
pd.DataFrame(history.history).plot(figsize = (12, 10));

In [ ]:
mod.evaluate(X_train_s, y_train)

In [ ]:
# Evaluate the model on the test set
pred = mod.predict(X_test_s)
pred[3]

In [ ]:
pred = np.argmax(pred, axis = 1)
pred.shape

In [ ]:
!rm 'submission.csv'
sub = pd.read_csv('../input/digit-recognizer/sample_submission.csv')
sub['Label'] = pred
sub.to_csv('submission.csv', index = False)
sub.head()